In [ ]:
import csv
import sys
import json

In [ ]:
maxInt = sys.maxsize

# workaround to load the csv file...https://stackoverflow.com/questions/15063936/csv-error-field-larger-than-field-limit-131072
while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)


In [ ]:
def load_nsc_csv(path, result = {}):
    first = True
    with open(path) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            if first or row[1] == "":
                first = False
                continue
            analysis_object = result.get(row[1], {})
            analysis_object["packageName"] = row[1]
            analysis_object["pinsNSC"] = row[4]
            analysis_object["pinPathsNSC"] = row[5]
            analysis_object["certsNSC"] = row[6]
            result[row[1]] = analysis_object

    return result

def load_string_csv(path, result = {}):
    first = True

    with open(path) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            if first or row[0] == "":
                first = False
                continue
            analysis_object = result.get(row[0], {})
            analysis_object["packageName"] = row[0]
            analysis_object["pinsString"] = row[3]
            analysis_object["pinPathsString"] = row[4]
            analysis_object["certsString"] = row[5]
            result[row[0]] = analysis_object


    return result

In [ ]:

gp_2022 = load_nsc_csv("certpinning/nsc/nsc_results_gp_2022.csv", {})


neupane = load_nsc_csv("certpinning/nsc/nsc_results_neupane.csv", {})
iotspotter = load_nsc_csv("certpinning/nsc/nsc_results_iotspotter.csv", {})
iotprofiler = load_nsc_csv("certpinning/nsc/nsc_results_iotprofiler.csv", {})


In [ ]:
gp_2022 = load_string_csv("certpinning/string_results/results_gp_2022.csv",gp_2022)

neupane = load_string_csv("certpinning/string_results/results_neupane.csv", neupane)
iotspotter = load_string_csv("certpinning/string_results/results_iotspotter.csv", iotspotter)
iotprofiler = load_string_csv("certpinning/string_results/results_iotprofiler.csv", iotprofiler)


In [ ]:
len(gp_2022)

In [ ]:
def get_mapping(file):
    with open(file) as f:
        return json.load(f)

In [ ]:
mapping_verified = get_mapping("verified_dataset/result.json")

In [ ]:
def get_only_latest(dataset, name, mapping):
    result = {}
    for k in dataset.keys():
        if k in mapping_verified:
            if mapping_verified[k] == name:
                result[k] = dataset[k]
        else:
            result[k] = dataset[k]

    return result




In [ ]:
iotspotter_removed = get_only_latest(iotspotter, "iotspotter",mapping_verified)


In [ ]:
iotprofiler_removed = get_only_latest(iotprofiler, "iotprofiler",mapping_verified)


In [ ]:
neupane_removed = get_only_latest(neupane, "neupane",mapping_verified)


In [ ]:
all_verified = iotspotter_removed | iotprofiler_removed | neupane_removed

In [ ]:
def remove_iot_apps_from_gp(gp_dict):
    manual = ["com.tuya.smartlife", "com.garmin.android.apps.connectmobile", "com.bshg.homeconnect.android.release", "com.huawei.health", "com.philips.ka.oneka.app", "com.google.android.apps.chromecast.app", "com.amazon.kindle", "jp.co.canon.bsd.ad.pixmaprint", "com.huami.watch.hmwatchmanager", "com.google.android.wearable.app", "com.sonos.acr2", "com.veryfit2hr.second", "com.amazon.dee.app", "com.aldi_project", "com.soomapps.screenmirroring", "com.irobot.home", "com.lgeha.nuts", "com.google.android.apps.fitness", "com.tuya.smart", "com.tonies.app", "com.hp.printercontrol", "com.garmin.connectiq", "com.crrepa.band.dafit", "jp.co.canon.android.printservice.plugin", "com.amazon.storm.lightning.client.aosp", "ir.remote.smg.tv", "com.vorwerk.cookidoo", "com.xiaomi.smarthome", "screen.mirroring.screenmirroring", "com.playstation.remoteplay", "jbl.stc.com", "com.tplink.iot", "com.strava", "com.hdvideoprojector.screenmirroring.castvideototv", "com.universal.tv.remote.control.all.tv.controller", "com.OnSoft.android.BluetoothChat", "plus.home_connect.android", "com.govee.home", "com.tplink.tether", "com.hp.android.printservice", "com.casttotv.screenmirroring.smarttv.castvideo", "soulapps.screen.mirroring.smart.view.tv.cast", "com.lscsmartconnection.smart", "at.a1.android.iapp", "epson.print", "com.sony.songpal.mdr", "com.hdvideoplayer.screencasting.simulator", "com.runtastic.android", "com.scee.psxandroid", "com.microsoft.xboxone.smartglass"]
    applist = ['com.OnSoft.android.BluetoothChat', 'com.amazon.storm.lightning.client.aosp', 'com.bshg.homeconnect.android.release', 'com.garmin.android.apps.connectmobile', 'com.google.android.apps.chromecast.app', 'com.google.android.apps.googleassistant', 'com.google.android.wearable.app', 'com.hp.android.printservice', 'com.huawei.health', 'com.irobot.home', 'com.lgeha.nuts', 'com.soundcloud.android', 'com.tplink.tether', 'com.tuya.smart', 'com.tuya.smartlife', 'com.xiaomi.hm.health', 'com.xiaomi.smarthome', 'epson.print', 'ir.remote.smg.tv', 'jp.co.canon.android.printservice.plugin']
    all_apps = set()
    all_apps.update(manual)
    all_apps.update(applist)
    for app in all_apps:
        if app in gp_dict:
            gp_dict.pop(app)

    return gp_dict




In [ ]:
gp_dict = remove_iot_apps_from_gp(gp_2022)

In [ ]:
def has_pin(object):
    return has_nsc_pin(object) or has_string_pin(object)

def has_nsc_pin(object):
    return object.get("pinsNSC", "[]") != "[]"

def has_string_pin(object):
    return  object.get("pinsString", "[]") != "[]"



def has_cert(object):
    return has_string_cert(object) or has_nsc_cert(object)


def has_string_cert(object):
    return object.get("certsString", "[]") != "[]"

def has_nsc_cert(object):
    return object.get("certsNSC", "[]") != "[]"


In [ ]:
def get_dataset_info(dataset):
    pins = 0
    cert = 0
    both = 0
    for k, app in dataset.items():
        if has_pin(app):
            pins = pins + 1

        if has_cert(app):
            cert = cert + 1

        if has_pin(app) and has_cert(app):
            both = both + 1
    return pins, cert, both

def get_dataset_info_nsc(dataset):
    pins = 0
    cert = 0
    both = 0
    for k, app in dataset.items():
        if has_nsc_pin(app):
            pins = pins + 1

        if has_nsc_cert(app):
            cert = cert + 1

        if has_nsc_pin(app) and has_nsc_cert(app):
            both = both + 1
    return [pins, cert, both]


def get_dataset_info_string(dataset):
    pins = 0
    cert = 0
    both = 0
    for k, app in dataset.items():
        if has_string_pin(app):
            pins = pins + 1

        if has_string_cert(app):
            cert = cert + 1

        if has_string_pin(app) and has_string_cert(app):
            both = both + 1
    return [pins, cert, both]


In [ ]:
rows = []
rows.append(get_dataset_info(all_verified))
rows.append(get_dataset_info(gp_2022))



In [ ]:
rows.append(get_dataset_info_nsc(all_verified))
rows.append(get_dataset_info_nsc(gp_2022))



In [ ]:
rows.append(get_dataset_info_string(all_verified))
rows.append(get_dataset_info_string(gp_2022))


In [ ]:
index_names = ["IoT-VER", "GP-2022", "IoT-VER-NSC","GP-2022-NSC", "IoT-VER-String","GP-2022-String"]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(rows, columns=['Pins', 'Cert', 'Both'], index = index_names)

In [ ]:
df

In [ ]:
import json
import re
import OpenSSL
import ssl
import base64
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization.pkcs7 import load_pem_pkcs7_certificates, load_der_pkcs7_certificates
from cryptography.hazmat.primitives import serialization

In [ ]:
def extract_certificates(dataset):
    result = []
    for _, app in dataset.items():
        appCerts = set()
        if "certsNSC" in app:
            appCerts = set(json.loads(app["certsNSC"]))


        certs = []
        if "certsString" in app:
            certs = (json.loads(app["certsString"]))

        for k in certs:
            appCerts.add(k)
        result.append(appCerts)
    return result



def extract_certificates_app(app):
    appCerts = set()
    if "certsNSC" in app:
        appCerts = set(json.loads(app["certsNSC"]))


    certs = []
    if "certsString" in app:
        certs = (json.loads(app["certsString"]))

    for k in certs:
        appCerts.add(k)
    return appCerts


In [ ]:
certs_all_verified = extract_certificates(all_verified)


In [ ]:
certs_neupane = extract_certificates(neupane_removed)
certs_iotspotter = extract_certificates(iotspotter_removed)
certs_iotprofiler = extract_certificates(iotprofiler_removed)


In [ ]:

certs_gp_2022 = extract_certificates(gp_2022)

In [ ]:
def parse_certs(possible_certificates):
    thisAppCerts = set()
    uniqueCertsStatic = set()
    successAllCerts = 0
    for c in possible_certificates:
            # Each entry itself might contain multiple certs if its PKCS7
        thisItemCertificates = []

        # We're dealing with a raw file (.crt, .pem, .der etc.)
        if "-----BEGIN" not in c:
            raw_file_bytes = base64.b64decode(c.encode())

            try:
                thisItemCertificates.append(x509.load_der_x509_certificate(raw_file_bytes, default_backend()))
                successAllCerts += 1
            except Exception as e:
                try:
                    thisItemCertificates = load_der_pkcs7_certificates(raw_file_bytes)
                    successAllCerts += 1
                except Exception as e:
                    try:
                        start = "-----BEGIN CERTIFICATE-----"
                        end = "-----END CERTIFICATE-----"
                        pem = start + "\n" + raw_file_bytes.decode() + "\n" + end
                        thisItemCertificates.append(
                            x509.load_pem_x509_certificate(pem.encode(), default_backend()))
                        successAllCerts += 1
                    except Exception as e:
                        try:
                            if raw_file_bytes.count(b"KEY-----") >= 1 or raw_file_bytes == b"":
                                successAllCerts += 1
                                continue

                            if raw_file_bytes.count(b"-----BEGIN CERTIFICATE-----") >= 1:
                                begins = [m.start() for m in re.finditer('-----BEGIN CERTIFICATE-----', raw_file_bytes.decode())]
                                ends = [m.start() for m in re.finditer('-----END CERTIFICATE-----', raw_file_bytes.decode())]
                                assert len(begins) == len(ends)
                                for i in range(0, len(begins)):
                                    # print(raw_file_bytes.decode()[begins[i]:ends[i]+len("-----END CERTIFICATE-----")])
                                    thisItemCertificates.append(
                                        x509.load_pem_x509_certificate(raw_file_bytes.decode()[begins[i]:ends[i]+len("-----END CERTIFICATE-----")].encode(), default_backend()))
                                successAllCerts += 1
                            else:
                                # We tried our best to parse the certificate, if its still unparsed, can't do better
                                assert False

                        except Exception as e:
                            print(e)
                            continue
                            # print(raw_file_bytes)
                            # input()

        # We're dealing with one BEGIN CERT/BEGIN PKCS7 string...
        else:

            assert c.count("-----BEGIN") == 1

            # Try loading PEM in the default way
            try:
                thisItemCertificates.append(x509.load_pem_x509_certificate(c.encode(), default_backend()))
                successAllCerts += 1
            except Exception as e:
                # Try fixing literal new lines
                try:
                    thisItemCertificates.append(x509.load_pem_x509_certificate(c.replace("\\n", "\n").replace("\\r", "\r").encode(), default_backend()))
                    successAllCerts += 1
                except Exception as e:
                    # Try fixing new lines at front and end
                    try:
                        thisItemCertificates.append(x509.load_pem_x509_certificate(
                            c.replace("-----BEGIN CERTIFICATE-----", "-----BEGIN CERTIFICATE-----\n").replace(
                            "-----END CERTIFICATE-----", "\n-----END CERTIFICATE-----").encode(), default_backend()))
                        successAllCerts += 1
                    except Exception as e:
                        # Try loading PKCS7 cert
                        try:
                            thisItemCertificates = load_pem_pkcs7_certificates(c.encode())
                            successAllCerts += 1
                        except Exception as e:
                            print(e)
                            continue

        for c_entry in thisItemCertificates:
            uniqueCertsStatic.add(c_entry.public_bytes(
                serialization.Encoding.PEM).decode())
            #print(uniqueCertsStatic)
            try:
                thisAppCerts.add(str(c_entry.subject))
                certificateSubjectToCertificate["static"][str(c_entry.subject)] = c_entry
            except:
                print("error parsing a certificate entry in static data...")
    return uniqueCertsStatic




In [ ]:
_PEM_RE = re.compile(b'-----BEGIN CERTIFICATE-----\r?.+?\r?-----END CERTIFICATE-----\r?\n?', re.DOTALL)


def parse_chain(file):
    with open(file, 'rb') as file_reader:
        chain = file_reader.read()
        # returns a list of certificates
        return [c.group() for c in _PEM_RE.finditer(chain)]

    return []

store = OpenSSL.crypto.X509Store()
file_path = ''
#OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, file_path)
for cert in parse_chain(file_path):
    store.add_cert(OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert))


In [ ]:
def is_expired(cert):
    x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert)

    return  x509.get_notAfter()

def get_year_month_date(byteDate):
    return byteDate[0:4], byteDate[4:6], byteDate[6:8]




def is_self_signed(cert):
    client_cert = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert)

    ctx = OpenSSL.crypto.X509StoreContext(store, client_cert)
    try:
        ctx.verify_certificate()
        return False
    except:
        print(cert)
        return True

In [ ]:
def parse_dataset(dataset):
    app_parsed_certs = []
    for certs in dataset:
        try:
            app_parsed_certs.append(parse_certs(certs))
        except:
            continue

    return app_parsed_certs




app_parsed_certs_neupane = parse_dataset(certs_neupane)
app_parsed_certs_iotspotter = parse_dataset(certs_iotspotter)
app_parsed_certs_iotprofiler = parse_dataset(certs_iotprofiler)

app_parsed_certs_all_verified = parse_dataset(certs_all_verified)

app_parsed_certs_gp_2022 = parse_dataset(certs_gp_2022)



In [ ]:
def has_expired_certs(parsed_dataset, year, month):
    apps_with_expired =0
    expired_certs = 0
    total_certs = 0

    for app in parsed_dataset:
        has_expired = False
        for c in app:
            total_certs = total_certs + 1
            y,m,d = get_year_month_date(is_expired(c))
            if y < year:
                has_expired = True
                expired_certs = expired_certs + 1
            elif y ==year and m < month:
                has_expired = True
                expired_certs = expired_certs + 1

        if has_expired:
            apps_with_expired = apps_with_expired + 1

    return [apps_with_expired, expired_certs, total_certs]

In [ ]:
def parse_iotspotter_file(path):
    first = True
    result = {}
    with open(path) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            if first or row[0] == "":
                first = False
                continue
            result[row[0]] = row[1]
    return result

def parse_androzoo_file(path):
    first = True
    result = {}
    with open(path) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            if first or row[0] == "":
                first = False
                continue
            result[row[0]] = row[10]
    return result

iotspotter_hashes = parse_iotspotter_file("shared_sha256_androzoo.csv")
androzoo_map = parse_androzoo_file("./latest_with-added-date.csv")





In [ ]:
def get_download_date_androzoo(id):
    hash_app = iotspotter_hashes.get(id)
    if hash_app is None:
        print(f"no hash for {id}")
        return "2021", "1", "1"
    date = androzoo_map.get(hash_app)
    if date is None:
        print(f"no date for {id}")
        return "2021", "1", "1"
    #print(date)
    splitted = date.split(" ")
    splitted = splitted[0].split("-")
    return splitted[0], splitted[1], splitted[2]

In [ ]:
def has_expired_certs_iotspotter(dataset):
    apps_with_expired =0
    expired_certs = 0
    total_certs = 0

    for appid, data in dataset.items():
        has_expired = False
        year, month, day =  get_download_date_androzoo(appid)

        certs = extract_certificates_app(data)
        try:
            certs = parse_certs(certs)
        except:
            continue
        for c in certs:
            total_certs = total_certs + 1
            y,m,d = get_year_month_date(is_expired(c))
            if int(y) < int(year):
                has_expired = True
                expired_certs = expired_certs + 1
            elif int(y) ==int(year) and int(m) < int(month):
                has_expired = True
                expired_certs = expired_certs + 1
            elif int(y) ==int(year) and int(m) == int(month) and int(d) < int(day):
                has_expired = True
                expired_certs = expired_certs + 1

        if has_expired:
            apps_with_expired = apps_with_expired + 1

    return [apps_with_expired, expired_certs, total_certs]

In [ ]:
def has_self_signed_certs(parsed_dataset):
    apps_with_self_signed =0
    self_signed = 0
    total_certs = 0

    for app in parsed_dataset:
        has_self_signed = False
        for c in app:
            total_certs = total_certs + 1
            if is_self_signed(c):
                has_self_signed = True
                self_signed = self_signed + 1
        if has_self_signed:
            apps_with_self_signed = apps_with_self_signed + 1

    return [apps_with_self_signed, self_signed, total_certs]

In [ ]:
def very_long_expiering_date(parsed_dataset, year, month):
    apps_with_long_expired =0
    long_expiering = 0
    total_certs = 0

    for app in parsed_dataset:
        has_very_long_expiering_date = False
        for c in app:
            total_certs = total_certs + 1
            y,m,d = get_year_month_date(is_expired(c))
            if y > year:
                has_very_long_expiering_date = True
                long_expiering = long_expiering + 1
            elif y ==year and m > month:
                has_very_long_expiering_date = True
                long_expiering = long_expiering + 1

        if has_very_long_expiering_date:
            apps_with_long_expired = apps_with_long_expired + 1

    return [apps_with_long_expired, long_expiering, total_certs]

In [ ]:
expired_iotspotter_exact = has_expired_certs_iotspotter(iotspotter_removed)
print("finish")

In [ ]:
expired_iotspotter_exact

In [ ]:
expired_iotspotter

In [ ]:
expired_neupane = has_expired_certs(app_parsed_certs_neupane, b'2021', b'1')
expired_iotprofiler = has_expired_certs(app_parsed_certs_iotprofiler, b'2018', b'1')
expired_iotspotter = has_expired_certs(app_parsed_certs_iotspotter, b'2021', b'1')

In [ ]:
expired_now_verified = has_expired_certs(app_parsed_certs_all_verified, b'2023', b'5')
expired_now_gp_2022 = has_expired_certs(app_parsed_certs_gp_2022, b'2023', b'5')

In [ ]:
expired_all_verified = has_expired_certs(app_parsed_certs_all_verified, b'2018', b'1')

In [ ]:
expired_gp_2022 = has_expired_certs(app_parsed_certs_gp_2022, b'2022', b'2')

In [ ]:
very_long_expiering_date(app_parsed_certs_all_verified, b'2032', b'2')

In [ ]:
very_long_expiering_date(app_parsed_certs_gp_2022, b'2032', b'2')

In [ ]:
self_all_verified = has_self_signed_certs(app_parsed_certs_all_verified)

In [ ]:
self_gp_2022 = has_self_signed_certs(app_parsed_certs_gp_2022)

In [ ]:
index_names_second_table = ["Apps Pinning", "Apps w/ Certs", "Both", "#Certs", "Apps w/ expired", "# expired",  "Apps w/ expired now", "# expired now","self signed", "# self signed"]

In [ ]:
general_gp_2022 =  get_dataset_info(gp_2022)

general_all_verified =  get_dataset_info(all_verified)

In [ ]:
rows_second_table = []

In [ ]:
rows_second_table.append([general_all_verified[0], general_gp_2022[0]])
rows_second_table.append([general_all_verified[1], general_gp_2022[1]])
rows_second_table.append([general_all_verified[2], general_gp_2022[2]])
rows_second_table.append([expired_all_verified[2], expired_gp_2022[2]])
rows_second_table.append([expired_neupane[0] + expired_iotspotter_exact[0] + expired_iotprofiler[0], expired_gp_2022[0]])
rows_second_table.append([expired_neupane[1] + expired_iotspotter_exact[1] + expired_iotprofiler[1], expired_gp_2022[1]])

rows_second_table.append([expired_now_verified[0], expired_now_gp_2022[0]])
rows_second_table.append([expired_now_verified[1], expired_now_gp_2022[1]])



rows_second_table.append([self_all_verified[0], self_gp_2022[0]])
rows_second_table.append([self_all_verified[1], self_gp_2022[1]])


In [ ]:
df = pd.DataFrame(rows_second_table, columns=['IoT-VER', "GP-2022"], index = index_names_second_table)

In [ ]:
df

In [ ]:
def format_row(df, column_name, rows, total_number):
    for i in rows:
        value = df[column_name].values[i]
        formatted_devision = f"{value} ({round(value*100/total_number, 2 )}%)"
        df.loc[df.index[i],column_name]= formatted_devision
        #print(formatted_devision)
        #print(df)
    return df

In [ ]:
df = format_row(df, 'IoT-VER', [0,1,2,4,6], 9889)

df = format_row(df, 'GP-2022', [0,1,2,4,6], 947)


In [ ]:
df = format_row(df, 'IoT-VER', [5,7], df.loc[df.index[3],'IoT-VER'])


df = format_row(df, 'GP-2022', [5,7], df.loc[df.index[3],'GP-2022'])


In [ ]:
df.loc[df.index[3],'GP-2022']

In [ ]:
df

In [ ]:
import statistics

In [ ]:
def getAverageAndStd(dataset):
    result = []
    for app in dataset:
        result.append(len(app))

    return f"{round(statistics.mean(result),2)} ({round(statistics.stdev(result), 2)})"

In [ ]:
new_row = [[getAverageAndStd(app_parsed_certs_all_verified), getAverageAndStd(app_parsed_certs_gp_2022)]]

In [ ]:
df_new = pd.DataFrame(new_row, columns=['IoT-VER', "GP-2022"], index = ["Avg (Std)"])

In [ ]:
df = pd.concat([df, df_new])

In [ ]:
iot_ver_column = df["IoT-VER"]


In [ ]:
df = df.drop(["IoT-VER"], axis=1)


In [ ]:
df["IoT-VER"] = iot_ver_column

In [ ]:
print(df.to_latex())

In [ ]:
df